<center>
    <tr>
    <td><img src="images/Quansight_Logo_Lockup_1.png" width="25%"></img></td>
    </tr>
</center>

# Camera Calibration in OpenCV

## Outline

- OpenCV checkerboard based camera calibration
- Image undistortion

Chessboards are frequently used as test images for camera calibration in computer vision.
+ [Chessboard detection](https://en.wikipedia.org/wiki/Chessboard_detection)
+ [OpenCV tutorial on camera calibration](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_calib3d/py_calibration/py_calibration.html)

The task here is to use utilities bundled with OpenCV to calibrate a camera from a set of chessboard images.

## Camera calibration using a checkerboard pattern

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

#### Exercise: loading a sequence of chessboard images

The directory `data` contains a set of images `left01.jpg`, `left02.jpg`, ... `left14.jpg` that can be used for camera calibration. Use a glob pattern to create a list `images` of all the image filepaths (i.e., a list of strings).

In [ ]:
# Your solution here

In [ ]:
# %load solutions/02/solution_01.py

#### Exercise: examining the first image

Extract the first image filename from the list `images` and convert it to a grayscale image.
+ Use [`cv.imread`](https://docs.opencv.org/2.4/modules/highgui/doc/reading_and_writing_images_and_video.html?highlight=imread#imread) to load the image.
+ Use [`cv.cvtColor`](https://docs.opencv.org/2.4/modules/imgproc/doc/miscellaneous_transformations.html#cvtcolor) to convert the image from RGB to grayscale.
+ Use the identifier `img` for the original image & `gray` for the grayscale version
+ Use `plt.imshow` to visualize the image `gray`.

In [ ]:
# Your solution here

In [ ]:
# %load solutions/02/solution_02.py

#### Exercise: examining the first image

+ Use the [`findChessboardCorners` function](https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html#findchessboardcorners) built-in to OpenCV to extract the corners from the image `gray`.
    + Assume a pattern of size `(9,6)` (corresponding to the interior corners to locate in the chessboard).
    + Assign the output array of corner coordinates to the identifier `corners`.
+ Use NumPy's `squeeze` function to eliminate singleton dimensions from the array `corners`.

We'll see later how corner detection is actually done.

In [ ]:
# Your solution here

In [ ]:
# %load solutions/02/solution_03.py

With the image `img` and the array `corners`, we can now produce a figure showing the original image and the image with circles overlaid on the corner coordinates.

In [ ]:
img2 = np.copy(img)  # Make a copy of original img as img2

# Add circles to img2 at each corner identified
for corner in corners:
    coord = (corner[0], corner[1])
    cv.circle(img=img2, center=coord, radius=5, color=(255, 0, 0), thickness=2)

# Produce a figure with the original image img in one subplot and modified image img2 (with the corners added in).
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(img)
plt.subplot(122)
plt.imshow(img2);

The [`cornerSubPix` function](https://docs.opencv.org/2.4/modules/imgproc/doc/feature_detection.html#cornersubpix) from OpenCV can be used to refine the corners extracted to sub-pixel accuracy. This is based on an iterative technique; as such, one of the inputs `criteria` uses a tuple to bundle a convergence tolerance and a maximum number of iterations.

In [ ]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001) # Set termination criteria as a tuple.
corners_orig = corners.copy()  # Preserve the original corners for comparison after
corners = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria=criteria) # extract refined corner coordinates.

In [ ]:
# Examine how much the corners have shifted (in pixels)
shift = corners - corners_orig
print(shift[:4,:])
print(np.linalg.norm(shift.reshape(-1,1), np.inf))

Now, generate a figure to compare the original corners to the corrected corners.

In [ ]:
img3 = np.copy(img)

for corner in corners:
    coord = (corner[0], corner[1])
    cv.circle(img=img3, center=coord, radius=5, color=(0, 255, 0), thickness=2)

plt.figure(figsize=(10,10))
plt.subplot(211)
plt.imshow(img2[200:300,200:400,:])
plt.subplot(212)
plt.imshow(img3[200:300,200:400,:]);

The function [`drawChessboardCorners`](https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html#drawchessboardcorners) generates a new image with circles at the corners detected. The corners are displayed either as red circles if the board was not found, or as colored corners connected with lines if the board was found (as determined by the output argument `retval` from `findChessboardCorners`).

In [ ]:
img4 = cv.drawChessboardCorners(img, (9, 6), corners, retval)
plt.figure(figsize=(10,10))
plt.imshow(img4);

Finally, we're going to repeat this process with all the chessboard images to remove distortion effects. First, assume a 3d world coordinate system aligned with the chessboard.

In [ ]:
obj_grid = np.zeros((9*6,3), np.float32)
obj_grid[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
print(obj_grid)

In [ ]:
# Initialize enpty list to accumulate coordinates
obj_points = [] # 3d world coordinates
img_points = [] # 2d image coordinates

In [ ]:
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)

for fname in images:
    print('Loading {}'.format(fname))
    img = cv.imread(fname)
    gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    
    retval, corners = cv.findChessboardCorners(gray, (9,6))
    if retval:
        obj_points.append(obj_grid)        
        corners2 = cv.cornerSubPix(gray, corners, (11,11), (-1,-1), criteria)
        img_points.append(corners2)

The accumulated lists of object coordinates and image coordinates can be combined to determine an optimal set of camera calibration parameters. The relevant OpenCV utility here is [`calibrateCamera`](https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html#cv2.calibrateCamera).

In [ ]:
retval, mtx, dist, rvecs, tvecs = cv.calibrateCamera(obj_points, img_points, gray.shape[::-1], None, None)

In [ ]:
print(retval) # Objective function value
print(mtx)    # Camera matrix
print(dist)   # Distortion coefficients

The function [`getOptimalNewCameraMatrix`](https://docs.opencv.org/2.4/modules/calib3d/doc/camera_calibration_and_3d_reconstruction.html#cv2.getOptimalNewCameraMatrix) can use the optimized matrix and distortion coefficients to construct a new camera matrix appropriate for a given image. This can be used to remove distortion effects with [`undistort`](https://docs.opencv.org/2.4/modules/imgproc/doc/geometric_transformations.html#undistort).

In [ ]:
img = cv.imread('data/left12.jpg')
h,w = img.shape[:2]
newcameramtx, roi = cv.getOptimalNewCameraMatrix(mtx, dist, (w,h), 1, (w,h))

In [ ]:
# undistort
dst = cv.undistort(img, mtx, dist, None, newcameramtx)

# crop the image
x,y,w,h = roi
dst = dst[y:y+h, x:x+w]

In [ ]:
plt.figure(figsize=(10,10))
plt.subplot(121)
plt.imshow(img)
plt.title('Original')
plt.subplot(122)
plt.imshow(dst)
plt.title('Corrected');

---
Based on materials from Prof. Faisal Qureshi (Faculty of Science, Ontario Tech University, Oshawa ON, Canada, http://vclab.science.ontariotechu.ca)

<center>
    <tr>
    <td><img src="images/Quansight_Logo_Lockup_1.png" width="25%"></img></td>
    </tr>
</center>